In [1]:
import pddlgym
import pddlparser, circuit

In [2]:
domain_fname = "cafeworld2.pddl"
problem_dir = "cafeworld_problem/"

# Get the model for this domain and problem
model1 = pddlparser.load_and_ground_pddl(domain_fname, problem_dir)
model2 = pddlparser.load_and_ground_pddl(domain_fname, problem_dir)

# Check if models are compatible; i.e., they have the same grounded predicates and actions (but each action has potentially different precondition or effect)
if pddlparser.are_models_compatible(model1, model2):
    print("Models are compatible!")
else:
    print("WARNING: Models are not compatible!")

# Map predicates to circuit variables
predicates_to_vars = {model1.predicates[i].pddl_str(): i for i in range(len(model1.predicates))}

Models are compatible!


In [3]:
# Takes initial state distribution, a list of PPDDL actions, and a model, and returns the final PC representing state distribution
def apply_action_sequence(initial_pc, action_sequence, model, predicates_to_vars, verbose=True):
    final_pc = initial_pc.copy()
    for t, a in enumerate(action_sequence):
        if verbose:
            print(f"Timestep t={t + 1}, action {a}")
        precondition = model.preconditions[a]
        effect = model.effects[a]

        final_pc = circuit.update_pc_with_action(final_pc, precondition, effect, predicates_to_vars)
    return final_pc

In [21]:
# We just want to initialize a distribution where the precondition holds for an action
action1 = 1

precondition1 = model1.preconditions[model1.actions[action1]]
effect1 = model1.effects[model1.actions[action1]]

action2 = 7

precondition2 = model1.preconditions[model1.actions[action2]]
effect2 = model1.effects[model1.actions[action2]]

In [22]:
effect2

PROBABILISTIC[(AND[Antiempty(gripper:manipulator), ingripper(canred:can,gripper:manipulator), Antiorder(canred:can,tablered:location)], 0.8), (AND[empty(gripper:manipulator)], 0.2), (NOCHANGE(), 0.0), (NOCHANGE(), 0.0)]

In [23]:
initial_state_1 = circuit.initial_state_pc({p.pddl_str(): True for p in precondition1.literals}, predicates_to_vars)
initial_state_2 = circuit.initial_state_pc({p.pddl_str(): True for p in precondition2.literals}, predicates_to_vars)

In [24]:
circuit.precondition_holds(initial_state_2, precondition2, predicates_to_vars)

True

In [25]:
u1 = apply_action_sequence(initial_state_1, [model1.actions[action1], model1.actions[2], model1.actions[3], model1.actions[4]] * 3, model1, predicates_to_vars)
u2 = apply_action_sequence(initial_state_2, [model2.actions[action2], model2.actions[5], model2.actions[6], model2.actions[4]] * 3, model2, predicates_to_vars)

Timestep t=1, action grasp(gripper:manipulator,counter:location,canred:can,fetch:robot)
Timestep t=2, action put(gripper:manipulator,tablered:location,canred:can,fetch:robot)
Timestep t=3, action move(tablered:location,counter:location,fetch:robot)
Timestep t=4, action teleport(tablered:location,fetch:robot)
Timestep t=5, action grasp(gripper:manipulator,counter:location,canred:can,fetch:robot)
Timestep t=6, action put(gripper:manipulator,tablered:location,canred:can,fetch:robot)
Timestep t=7, action move(tablered:location,counter:location,fetch:robot)
Timestep t=8, action teleport(tablered:location,fetch:robot)
Timestep t=9, action grasp(gripper:manipulator,counter:location,canred:can,fetch:robot)
Timestep t=10, action put(gripper:manipulator,tablered:location,canred:can,fetch:robot)
Timestep t=11, action move(tablered:location,counter:location,fetch:robot)
Timestep t=12, action teleport(tablered:location,fetch:robot)
Timestep t=1, action grasp(gripper:manipulator,tablered:location,ca

In [29]:
import time

start = time.time()
dist = circuit.cw_dist(u1, u2)
end = time.time()
print(f"Distance of {dist} computed in {end - start} seconds!")

Distance of 5.117721569689601 computed in 7.394667625427246 seconds!


In [30]:
u2.size()

2221

In [31]:
# Get the (approximate) most likely satisfying assignment for u1
u1.map()

{0: 1, 1: 0, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1}

In [32]:
u1.size()

2203